In [ ]:
import pandas as pd
import numpy as np
import calendar
from datetime import timedelta
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('2021_2022_2023_expense.csv')

df_dd = df[df.CATEGORIES == 'DryDocking Expenses'].reset_index(drop=True)
df_pd = df[df['CATEGORIES']=='Pre-Delivery Expenses'].reset_index(drop=True)

print(df_dd.shape)

In [ ]:
vessel_particulars = pd.read_excel('VESSEL_PARTICULARS.xlsx')
# vessel_particulars[['VESSEL TYPE', 'VESSEL SUBTYPE']].value_counts().reset_index()['VESSEL SUBTYPE'].unique()

In [ ]:
def generate_segments(dates_series, interval_years=5, interval_months=0):
    segments = []
    dates_series.sort_values(inplace=True)
    current_date = dates_series.iloc[0]  # Start with the minimum date
    
    for date in dates_series.iloc[1:]:
        segment_end = current_date + timedelta(
            days=(interval_years * 365.25 + interval_months * 30.44) - 1
        )
        if date > segment_end:
            segments.append((current_date, segment_end))
            current_date = date
    segments.append((current_date, dates_series.iloc[-1]))  # Last segment
    return segments

In [ ]:
def func(x):
    dates = pd.to_datetime(x['DATE']).copy()
    segments = generate_segments(dates)
    return segments

In [ ]:
def get_aggregation(segments, flag='cat'):
    results = []
    
    if flag == 'subcat':
        for x in segments.reset_index(name='segments')[['COST_CENTER', 'segments', 'CATEGORIES', 'ACCOUNT_CODE', 'SUB_CATEGORIES']].values:
            for dt in x[1]:
                temp = df_dd[df_dd.COST_CENTER == x[0]]
                temp['DATE'] = pd.to_datetime(temp['DATE'])
                expense = temp[(temp['DATE'] >= x[1][0][0]) & (temp['DATE'] == x[1][0][1])]['AMOUNT_USD'].sum()
                results.append((x[0], x[1][0], expense, x[2], x[3], x[4]))
    else:
        for x in segments.reset_index(name='segments')[['COST_CENTER', 'segments', 'CATEGORIES']].values:
            for dt in x[1]:
                temp = df_dd[df_dd.COST_CENTER == x[0]]
                temp['DATE'] = pd.to_datetime(temp['DATE'])
                expense = temp[(temp['DATE'] >= x[1][0][0]) & (temp['DATE'] == x[1][0][1])]['AMOUNT_USD'].sum()
                results.append((x[0], x[1][0], expense, x[2]))
    
    return results

In [ ]:
def get_pd_data(df_pd):
    cat_df_pd = df_pd.groupby(['COST_CENTER', 'CATEGORIES']).AMOUNT_USD.median()

    subcat_df_pd = df_pd.groupby(['COST_CENTER', 'CATEGORIES', 'ACCOUNT_CODE', 'SUB_CATEGORIES']).AMOUNT_USD.median()
    
    cat_df_pd = cat_df_pd.groupby(['CATEGORIES']).agg(
        q1=lambda x: np.quantile(x, 0.25),
        q2=lambda x: np.quantile(x, 0.50),
        median=lambda x: np.quantile(x, 0.63),
        q3=lambda x: np.quantile(x, 0.75)
        ).astype(int)
    
    subcat_df_pd = subcat_df_pd.groupby(['CATEGORIES', 'ACCOUNT_CODE', 'SUB_CATEGORIES']).agg(
        q1=lambda x: np.quantile(x, 0.25),
        q2=lambda x: np.quantile(x, 0.50),
        median=lambda x: np.quantile(x, 0.63),
        q3=lambda x: np.quantile(x, 0.75)
        ).astype(int)
    
    return cat_df_pd, subcat_df_pd

In [ ]:
def get_dd_cat(DF_DD):
    cost_centers = []
    expenses = []
    
    df_dd_cat = DF_DD.groupby(['COST_CENTER', 'PERIOD', 'CATEGORIES']).AMOUNT_USD.sum().reset_index()
    df_dd_cat['DATE'] = df_dd_cat['PERIOD'].astype('str').apply(lambda x: f"{x[:4]}-{x[4:]}-01" if x else None)
    # df_dd_cat.groupby(['COST_CENTER', 'CATEGORIES', 'PERIOD', 'DATE'])['AMOUNT_USD'].sum().reset_index()
    cat_seg = df_dd_cat.groupby(['COST_CENTER', 'CATEGORIES']).apply(func)

    for rec in cat_seg.reset_index(name='daterange').itertuples():
        cc = rec[1]
        for dd in rec[3]:
            temp = df_dd_cat[(df_dd_cat.COST_CENTER == cc) & (df_dd_cat.DATE >= pd.to_datetime(dd[0]).strftime("%Y-%m-%d")) & (df_dd_cat.DATE <= pd.to_datetime(dd[1]).strftime("%Y-%m-%d"))]
            cost_centers.append(cc)
            expenses.append(temp.AMOUNT_USD.sum())
            
    cat_seg_event = pd.DataFrame()
    cat_seg_event['COST_CENTER'] = pd.Series(cost_centers)
    cat_seg_event['EXPENSE'] = pd.Series(expenses)

    filtered_df = cat_seg_event[cat_seg_event.EXPENSE != 0.00]
    q1 = filtered_df['EXPENSE'].quantile(0.25)
    q2 = filtered_df['EXPENSE'].quantile(0.50)  # This is the median
    q3 = filtered_df['EXPENSE'].quantile(0.75)
    

    # Create a DataFrame with quartile values
    return pd.DataFrame({'Quartile': ['CATEGORIES', 'median_50perc_population', 'optimal_63perc_population', 'top_75perc_population'],
                                'Value': [rec[2], q1, q2, q3]})
    
dd_cat = get_dd_cat(df_dd)

In [ ]:
def get_dd_subcat(DF_DD):
    cost_centers = []
    expenses = []
    ac_codes=[]
    sub_cats = []

    df_dd_cat = DF_DD.groupby(['COST_CENTER', 'PERIOD', 'CATEGORIES', 'ACCOUNT_CODE', 'SUB_CATEGORIES']).AMOUNT_USD.sum().reset_index()
    df_dd_cat['DATE'] = df_dd_cat['PERIOD'].astype('str').apply(lambda x: f"{x[:4]}-{x[4:]}-01" if x else None)
    # df_dd_cat.groupby(['COST_CENTER', 'CATEGORIES', 'PERIOD', 'DATE'])['AMOUNT_USD'].sum().reset_index()
    cat_seg = df_dd_cat.groupby(['COST_CENTER', 'CATEGORIES', 'ACCOUNT_CODE', 'SUB_CATEGORIES']).apply(func)

    for rec in cat_seg.reset_index(name='daterange').itertuples():
        cc = rec[1]
        for dd in rec[5]:
            temp = df_dd_cat[(df_dd_cat.COST_CENTER == cc) & (df_dd_cat.DATE >= pd.to_datetime(dd[0]).strftime("%Y-%m-%d")) & (df_dd_cat.DATE <= pd.to_datetime(dd[1]).strftime("%Y-%m-%d"))]
            cost_centers.append(cc)
            expenses.append(temp.AMOUNT_USD.sum())
            ac_codes.append(rec[3])
            sub_cats.append(rec[4])
            
    subcat_seg_event = pd.DataFrame()
    subcat_seg_event['COST_CENTER'] = pd.Series(cost_centers)
    subcat_seg_event['CATEGORIES'] = rec[2]
    subcat_seg_event['ACCOUNT_CODE'] = pd.Series(ac_codes)
    subcat_seg_event['SUB_CATEGORIES'] = pd.Series(sub_cats)
    subcat_seg_event['EXPENSE'] = pd.Series(expenses)

    filtered_df = subcat_seg_event[subcat_seg_event.EXPENSE != 0.00]

    subcat_df_pd = filtered_df.groupby(['CATEGORIES', 'ACCOUNT_CODE', 'SUB_CATEGORIES']).agg(
        median_50perc_population=('EXPENSE', lambda x: np.quantile(x, 0.50)),
        optimal_63perc_population=('EXPENSE', lambda x: np.quantile(x, 0.63)),
        top_75perc_population=('EXPENSE', lambda x: np.quantile(x, 0.75))
        ).astype(int)

    return subcat_df_pd
    
dd_subcat = get_dd_subcat(df_dd)

In [ ]:
dd_cat = dd_cat.head().set_index('Quartile').T.reset_index(drop=True)
dd_subcat = dd_subcat.reset_index()

In [ ]:
pd_cat, pd_subcat =  get_pd_data(df_pd)

In [ ]:
pd_cat.reset_index()

In [ ]:
dd_cat

## 06/20/2024

In [ ]:
import pandas as pd
import numpy as np
import json

In [ ]:
event_df = pd.read_excel('event_excel.xlsx')

event_df['median_50perc_population'] = event_df['median_50perc_population'].astype(int)
event_df['optimal_63perc_population'] = event_df['optimal_63perc_population'].astype(int)
event_df['top_75perc_population'] = event_df['top_75perc_population'].astype(int)

grp_tot_cat = event_df.groupby(['CATEGORIES']).sum().reset_index().sum().to_dict()


# condition = grp_tot_cat["Header"].isin(['Total OPEX', 'OPEX/DAY'])

# if condition.shape[0] > 0:
#     grp_tot_cat.loc[condition, 'Stats Model - Optimal Budget'] = None
#     grp_tot_cat.loc[condition, 'median_50perc_population'] = None
#     grp_tot_cat.loc[condition, 'optimal_63perc_population'] = None
#     grp_tot_cat.loc[condition, 'top_75perc_population'] = None

# grp_tot_cat['order'] = grp_tot_cat['Header'].apply(lambda x: order.index(x) if x in order else len(order))
# grp_tot_cat = grp_tot_cat.sort_values(by='CATEGORIES', ascending=True)
# grp_tot_cat = grp_tot_cat.set_index('CATEGORIES')        
# grp_tot_cat = grp_tot_cat.sort_values(by='CATEGORIES', ascending=True)

# grp_tot_cat = grp_tot_cat.T.to_json()

# ## cate vise sum
json_data = {}

# # Group by 'Header' column
grouped = event_df.groupby(['CATEGORIES'])

# # Iterate over groups
for group_name, group_data in grouped:
    group_json = group_data.to_dict(orient='records')
    json_data[group_name] = group_json
    json_output = json.dumps(json_data, indent=4)

# grp_tot_cat = json.loads(grp_tot_cat)
# for key in grp_tot_cat.keys():
#     grp_tot_cat[key].update({"records": json_data[key]})
#     grp_tot_cat[key].update({"Header": key})                
    

# data = []
# for key, value in grp_tot_cat.items():
#     data.append(grp_tot_cat[key])

In [ ]:
event_df['median_50perc_population'] = event_df['median_50perc_population'].astype(int)
event_df['optimal_63perc_population'] = event_df['optimal_63perc_population'].astype(int)
event_df['top_75perc_population'] = event_df['top_75perc_population'].astype(int)

grp_tot_cat = event_df.groupby(['CATEGORIES']).sum().reset_index().sum().to_dict()
grp_tot_cat['CATEGORIES'] = 'EVENT CATEGORIES'
grp_tot_cat

In [ ]:
import warnings
warnings.filterwarnings('ignore')

event_df = pd.read_excel('event_excel.xlsx')

event_df['median_50perc_population'] = event_df['median_50perc_population'].astype(int)
event_df['optimal_63perc_population'] = event_df['optimal_63perc_population'].astype(int)
event_df['top_75perc_population'] = event_df['top_75perc_population'].astype(int)

event_df['SUBCATEGORIES'] = event_df['ACCOUNT_CODE'] + "; " + event_df['SUB_CATEGORIES']
final_json = event_df[['CATEGORIES', 'median_50perc_population', 'optimal_63perc_population', 'top_75perc_population']].groupby('CATEGORIES').sum().T.to_dict()

dd_recs = []
pd_recs = []

grouped = event_df[['CATEGORIES', 'SUBCATEGORIES', 'median_50perc_population', 'optimal_63perc_population', 'top_75perc_population']].rename(columns={'CATEGORIES':'Header'}).groupby('Header')

# Iterate over groups and convert each group to a dictionary
for group_name, group_data in grouped:
    group_dict = {
        'CATEGORY': group_name,
        'data': group_data.to_dict(orient='records')
    }

    item_k = list(group_dict.keys())
    
    if group_dict[item_k[0]] == 'DRYDOCKING EXPENSES':
        dd_recs.extend(group_dict[item_k[1]])
    elif group_dict[item_k[0]] == 'PRE-DELIVERY EXPENSES':
        pd_recs.extend(group_dict[item_k[1]])
        
final_json['DRYDOCKING EXPENSES'].update({'records': dd_recs})
final_json['PRE-DELIVERY EXPENSES'].update({'records' : pd_recs}) 
recs = list(final_json.values())

event_df = event_df.groupby(['CATEGORIES']).sum().reset_index().sum().to_dict()
event_df['CATEGORIES'] = 'TOTAL EVENT EXPENSE'
event_df['Header'] = 'TOTAL EVENT EXPENSE'

recs.append(event_df)

with open("valid.json", 'w') as f:
    json.dump(recs, f, indent=4)

In [ ]:
from dotenv import load_dotenv
import os
import snowflake.connector
import pandas as pd
import datetime

load_dotenv()

account = os.getenv('account')
user = os.getenv('user')
password = os.getenv('password')
warehouse = os.getenv('warehouse')
database = os.getenv('database')
schema = os.getenv('schema')

In [ ]:
def get_data(query, database=None, schema=None):
    # Establish the connection
    conn = snowflake.connector.connect(
        user=user,
        password= password,
        account= account,
        warehouse= warehouse,
        database= database,
        schema= schema
    )

    # Create a cursor object
    cursor = conn.cursor()

    cursor.execute(query)

    # Fetch and print the result
    result = cursor.fetchall()

    # Close the cursor and connection
    cursor.close()
    conn.close()
    return result

In [ ]:
def get_expense_data():
    unique_cc = pd.read_excel('VESSEL_PARTICULARS.xlsx', engine='openpyxl')['VESSEL CODE'].unique().tolist()
    unique_cc_str = ', '.join([f"'{cc}'" for cc in unique_cc])
    
    import datetime
    # Get the current year
    current_year = datetime.datetime.now().year

    # Calculate the start and end periods
    start_period = (current_year - 3) * 100 + 1  # January of the year 3 years ago
    end_period = (current_year - 1) * 100 + 12  # December of last year        

    expense_query = f'''
    SELECT
        TO_VARIANT(DATE_TRUNC('MONTH', MS.POSTING_DATE))::VARCHAR AS "Year-Month",
        MS.COST_CENTER,
        MS.NODECODE AS "Categories",
        MS.ACCOUNT_CODE,
        MS.ACCOUNT_CODE_DESC AS "SUB_CATEGORIES",
        SUM(MS.AMOUNT_USD) AS "Expense"
    FROM
        CURATED_DB.VESSEL_ACCOUNTS.MANAGER_STATEMENT MS
    WHERE
        MS.BRANCHCODE = 'Operating Expenses' AND 
        MS.COST_CENTER IN ({unique_cc_str}) AND 
        MS.PERIOD >= {start_period} AND 
        MS.PERIOD <= {end_period}
    GROUP BY 
        "Year-Month", 
        MS.COST_CENTER, 
        MS.NODECODE, 
        MS.ACCOUNT_CODE, 
        MS.ACCOUNT_CODE_DESC
    ORDER BY 
        "Year-Month", 
        MS.COST_CENTER, 
        MS.NODECODE ASC
    '''

    expense_data = pd.DataFrame(get_data(expense_query), columns=['DATE', 'COST_CENTER', 'CATEGORIES', "ACCOUNT_CODE", 'SUB_CATEGORIES', 'Expense'])
    expense_data['PERIOD'] = pd.to_datetime(expense_data['DATE']).dt.strftime('%Y%m').astype('int')
    expense_data = expense_data[(expense_data['PERIOD'] >= start_period) & (expense_data['PERIOD'] <= end_period)]
    
    return expense_data

In [ ]:
data = get_expense_data()

In [ ]:
data.DATE.min(), data.DATE.max()

202101 202312
